In [1]:
!python3 -m pip install nextstrain-augur

Defaulting to user installation because normal site-packages is not writeable


In [86]:
import json
from augur.utils import json_to_tree

In [87]:
json_fh = open("/Users/Carlos/Downloads/h3n2_60y_pb2.json", "r")
json_dict = json.load(json_fh)
tree = json_to_tree(json_dict)

In [88]:
pb2_mutations_dict = {}

for node in tree.find_clades():
    if not node.branch_attrs.get('mutations'):
        continue

    if not node.branch_attrs['mutations'].get('PB2'):
        continue

    # Retrieve PB2 mutations
    pb2_mutations = node.branch_attrs['mutations'].get('PB2')
    
    # Add PB2 mutations to the dictionary
    for mutation in pb2_mutations:
        if mutation not in pb2_mutations_dict:
            pb2_mutations_dict[mutation] = []
        # Add terminal names associated with this mutation
        terminal_names = [tip.name for tip in node.get_terminals()]
        pb2_mutations_dict[mutation].extend(terminal_names)

In [89]:
#pb2_mutations_dict

In [90]:
json_fh_frequency = open("/Users/Carlos/Downloads/h3n2_60y_pb2_tip-frequencies.json", "r")
json_dict_frequency = json.load(json_fh_frequency)
#json_dict_frequency

In [91]:
# Update pb2_mutations_dict to include maximum frequencies
for mutation, terminal_names in pb2_mutations_dict.items():
    for i, terminal_name in enumerate(terminal_names):
        # Check if the terminal name exists in max_frequency_per_terminal
        if terminal_name in max_frequency_per_terminal:
            # Add the maximum frequency to the terminal name entry
            max_frequency = max_frequency_per_terminal[terminal_name]
            # Update the terminal name with its frequency in pb2_mutations_dict
            terminal_names[i] = { "name": terminal_name, "max_frequency": max_frequency }
        else:
            # If no frequency is found, keep the terminal name as is
            terminal_names[i] = { "name": terminal_name, "max_frequency": None }

# Print the updated pb2_mutations_dict
#print("Updated pb2_mutations_dict with Maximum Frequencies:")
#print(pb2_mutations_dict)

In [92]:
#pb2_mutations_dict

In [93]:
# Dictionary to store the maximum frequency for each mutation
max_frequency_per_mutation = {}

for mutation, terminal_names in pb2_mutations_dict.items():
    max_frequency = 0  # Initialize the maximum frequency for the mutation
    for terminal in terminal_names:
        # Extract the terminal name if it's a dictionary
        if isinstance(terminal, dict):
            terminal_name = terminal.get("name")
        else:
            terminal_name = terminal  # Assume it's a string if not a dictionary

        # Check if the terminal name exists in max_frequency_per_terminal
        if terminal_name in max_frequency_per_terminal:
            frequency = max_frequency_per_terminal[terminal_name]
            # Update max_frequency if the current frequency is larger
            if frequency > max_frequency:
                max_frequency = frequency
    # Store the maximum frequency for the mutation
    max_frequency_per_mutation[mutation] = max_frequency

# Print the resulting dictionary
#print("Maximum Frequency for Each Mutation:")
#print(max_frequency_per_mutation)

In [94]:
#max_frequency_per_mutation

In [95]:
# Export max_frequency_per_mutation to a text file
output_file_path = "/Users/Carlos/Desktop/Bedford/esm-selection/max_frequency_per_mutation.txt"

with open(output_file_path, "w") as file:
    for mutation, max_frequency in max_frequency_per_mutation.items():
        file.write(f"{mutation}: {max_frequency}\n")

# Get frequencies 

### Activate nextstrain docker container

``` bash
nextstrain shell .
```

### Get max frequencies using Augur

PB2/masked.gene.PB2_Edited.fasta file was edited because "EPI381962 Influenza A virus (A/Aichi/2/1968-egg) segment 2 polymerase (PB2), complete cds" was not in metadata rather A/Aichi/2/1968-egg was 

``` bash
augur frequencies --method diffusion --metadata PB2/metadata.tsv --alignments PB2/masked.gene.PB2_Edited.fasta --gene-names PB2 --ignore-char X --minimal-frequency 0.01 --pivot-interval 3 --pivot-interval-units months --output PB2_frequencies.json
```

"Optimization failed, trying with powell"

Error was spit out but still generated frequencies json



In [5]:
# Import necessary libraries
import json
import csv

# Load the JSON file
json_file_path = "/Users/Carlos/Desktop/Bedford/esm-selection/PB2_frequencies.json"  # Update with your JSON file path
with open(json_file_path, "r") as json_file:
    json_data = json.load(json_file)

# Dictionary to store the maximum value for each key
max_values = {}

# Iterate through the JSON data to find the maximum value for each key
for key, values in json_data.items():
    if isinstance(values, list) and values:  # Ensure the value is a non-empty list
        max_values[key] = max(values)

# Export the max values to a CSV file
csv_output_file_path = "/Users/Carlos/Desktop/Bedford/esm-selection/max_values.csv"
with open(csv_output_file_path, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write the header row
    csv_writer.writerow(["Key", "Max Value"])
    # Write each key and its max value
    for key, max_value in max_values.items():
        csv_writer.writerow([key, max_value])

print(f"Max values exported to {csv_output_file_path}")

Max values exported to /Users/Carlos/Desktop/Bedford/esm-selection/max_values.csv
